## <center> Linear Transformations of a PlotlyJS figure(plot)</center>

Here we present another context when `PlotlyJS.image` is used. 
Namely, if  an arbitrary  PlotlyJS plot need to be rotated or transformed by any linear map, 
then the corresponding plot is converted to a `Matrix{RGBA{N0f8}}` (the default type returned by the function `fig2array`, defined below),
and this matrix, to a Base64 string, which is passed as `source` for a PlotlyJS.image.

In [ ]:
using  Rotations, ImageTransformations, CoordinateTransformations,  FileIO, Images
using Base64, PlotlyJS

function fig2array(fig; width=500, height=500, scale=1 )
    #convert a Plotly plot(figure) to  a  Matrix{RGBA{N0f8}}
    io = IOBuffer()
    savefig(io, fig; width=width, height=height, scale=scale, format="png")
    strm = Stream(format"PNG", io)
    load(strm); 
end   

bs64string(img) = "data:image/png;base64,$(stringmime(MIME("image/png"), img))"

pl_imshow(img)= Plot(PlotlyJS.image(; source=bs64string(img), hoverinfo="none"))

axes_off(fig) = relayout!(fig, xaxis_visible=false, #remove axes 
                               yaxis_visible=false);



### Heatmap plot rotation

In [ ]:
x= collect(-5:0.075:10)
y= collect(-5:0.075:5)
z = [(sin(xl)^10 + cos(10+yl*xl) + cos(xl) + 0.2*yl + 0.1 * xl) for yl in y, xl in x];

In [ ]:
fig = Plot(heatmap(x= x, 
                   y= y,
                   z= z,
                   colorscale=colors.curl, colorbar_thickness=23))
relayout!(fig, width=400, height=375)
#display(fig)
farr = fig2array(fig;)
rot = recenter(RotMatrix(pi/4), ImageTransformations.center(farr))
#the below conversion, RGB.(farr), is needed because arr=load(strm) is a Matrix{RGBA{N0f8}} 
#and ImageTransformations works with Matrix{RGB{T}} type matrices
imgw = warp(RGB.(farr), rot, fillvalue=1);


Let us create a subplot of two cells to display both the original plot and the rotated one:

In [ ]:
figs = make_subplots(rows=1, cols=2, subplot_titles=["Initial plot" "Rotated plot"],
                     )
add_trace!(figs, PlotlyJS.image(; source=bs64string(farr), hoverinfo="none"), row=1, col=1)
add_trace!(figs, PlotlyJS.image(; source=bs64string(imgw), hoverinfo="none"), row=1, col=2)
relayout!(figs, width=800, height=500, xaxis_visible=false, yaxis_visible=false, 
          xaxis2_visible=false, yaxis2_visible=false, xaxis_domain=[0, 0.4],
    xaxis2_domain=[0.43,1])
display(figs)

Remark: If the `hoverinfo="none"` is removed from the definition of the two traces,
at hover is displayed the colorant, RGB, of the corresponding pixel, not the value mapped to that color. This is the information displayed by default for `PlotlyJS.image` traces.

### Heatmap matrix rotation

Instead of rotating the entire heatmap plot we can rotate only the matrix  z:

In [ ]:
n = 2^8
I = [i for i in 0:n-1, j in 0:n-1]
z = xor.(I', I)
figh= Plot(heatmap(z=z, colorscale=colors.plasma, colorbar_thickness=23), 
           Layout(width=450, height=425, font_family="Balto"))
display(figh)

In [ ]:
rot = recenter(RotMatrix(-pi/4), ImageTransformations.center(z));
zr = warp(z, rot, fillvalue=NaN) 

fig1 = Plot(heatmap(z=zr, colorscale=colors.plasma,  colorbar_thickness=23,
                   hovertemplate="z: %{z:d}<extra></extra>",
                   hoverongaps=false), 
            Layout(width=500, height=500, font_family="Balto",
                  yaxis_scaleratio=1, yaxis_scaleanchor="x",
                  plot_bgcolor="rgba(0,0,0,0)"))
axes_off(fig1)
display(fig1)

Rotating only the heatmap matrix, z, the corresponding heatmap plot is  still interactive.